### Task 2 -  Data Modeling and Transformation (Transform)

In [23]:
# %pip install python-dotenv psycopg2-binary

In [42]:
import os
import sys
import json
import psycopg2
from psycopg2.extras import execute_values
import shutil
from datetime import datetime
sys.path.append(os.path.abspath("../Script"))
import warnings
warnings.filterwarnings('ignore')

In [14]:
os.chdir("..")
print(os.getcwd()) 

d:\KAIM 5,6,7\Week-7\Shipping-Data-Product


In [24]:
from connection import getpgsql_connect 

In [62]:
def ensure_table_exists(cursor, conn):
    cursor.execute("""
        CREATE SCHEMA IF NOT EXISTS raw;

        CREATE TABLE IF NOT EXISTS raw.raw_telegram_messages (
            message_id INT,
            sender_id TEXT,
            date TIMESTAMP,
            text TEXT,
            channel TEXT,
            media_type TEXT
        );
    """)
    conn.commit()  # ✅ Ensure table creation is persisted

In [63]:
def load_json_to_postgres(json_path, table_name='raw.raw_telegram_messages'):

    try:
        conn = getpgsql_connect()
        cursor = conn.cursor()

        #  Ensure table exists and COMMIT
        ensure_table_exists(cursor, conn)

        # Load JSON
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        rows = [
            (
                msg.get('id'),
                msg.get('sender_id'),
                msg.get('date'),
                msg.get('text'),
                msg.get('channel'),
                msg.get('media_type')
            )
            for msg in data
        ]

        #  Insert
        insert_query = f"""
            INSERT INTO {table_name} 
            (message_id, sender_id, date, text, channel, media_type)
            VALUES %s
        """
        execute_values(cursor, insert_query, rows)

        conn.commit()
        print(f" Inserted {len(rows)} records into {table_name}")
        cursor.close()
        conn.close()

    except Exception as e:
        print(f" Error inserting data: {e}")


In [64]:
json_path = "Data/Raw/Telegram_Med_messages/2025-07-15/lobelia4cosmetics.json"
load_json_to_postgres(json_path)

 Inserted 20 records into raw.raw_telegram_messages


In [ ]:
def ensure_table_exists(cursor, conn):
    cursor.execute("""
        CREATE SCHEMA IF NOT EXISTS raw;

        CREATE TABLE IF NOT EXISTS raw.raw_telegram_messages (
            message_id INT,
            sender_id TEXT,
            date TIMESTAMP,
            text TEXT,
            channel TEXT,
            media_type TEXT
        );
    """)
    conn.commit()

def load_json_to_postgres(json_path, table_name='raw.raw_telegram_messages'):
    # import dotenv
    # dotenv.load_dotenv()

    try:
        conn   = getpgsql_connect()
        cursor = conn.cursor()

        # Ensure table and schema
        ensure_table_exists(cursor, conn)

        # Load JSON
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        rows = []
        images_saved = 0

        for msg in data:
            rows.append((
                msg.get('id'),
                msg.get('sender_id'),
                msg.get('date'),
                msg.get('text'),
                msg.get('channel'),
                msg.get('media_type')
            ))

            # Handle image saving (if path exists)
            media_path = msg.get("media_file_path")
            # media_path = 'Data/Raw/Telegram_Med_messages/2025-07-15/images'
            if media_path and os.path.exists(media_path):
                channel = msg.get('channel').replace('@', '')
                date_part = datetime.now().strftime('%Y-%m-%d')
                dest_dir = f"data/images/telegram/{date_part}/{channel}"
                os.makedirs(dest_dir, exist_ok=True)
                dest_path = os.path.join(dest_dir, os.path.basename(media_path))

                shutil.copy(media_path, dest_path)
                images_saved += 1

        # Insert data into PostgreSQL
        insert_query = f"""
            INSERT INTO {table_name}
            (message_id, sender_id, date, text, channel, media_type)
            VALUES %s
        """
        execute_values(cursor, insert_query, rows)
        conn.commit()

        print(f"Inserted {len(rows)} records into {table_name}")
        print(f"Saved {images_saved} image files")

        cursor.close()
        conn.close()

    except Exception as e:
        print(f"Error inserting data or saving images: {e}")

In [66]:
json_path = "Data/Raw/Telegram_Med_messages/2025-07-15/lobelia4cosmetics.json"
load_json_to_postgres(json_path)

Inserted 20 records into raw.raw_telegram_messages
Saved 0 image files


In [67]:
print(f" Connecting to PostgreSQL database {os.getenv('pgsql_db')}")
...
print(f" Processing JSON file: {json_path}")
print(f"Found {len(data)} messages to insert")
...
print(f"Copied image to {dest_path}")


 Connecting to PostgreSQL database telegram
 Processing JSON file: Data/Raw/Telegram_Med_messages/2025-07-15/lobelia4cosmetics.json


NameError: name 'data' is not defined